In [1]:
# Useful starting lines
%matplotlib inline

import random
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

# Support Vector Machines
## Classification Using SVM
Load dataset. We will re-use the CERN dataset from project 1, available from https://inclass.kaggle.com/c/epfml-project-1/data

In [2]:
from helpers import load_csv_data

DATA_TRAIN_PATH = 'data/train.csv'

y, X, ids = load_csv_data(DATA_TRAIN_PATH, sub_sample=True)
print(y.shape, X.shape)

(5000,) (5000, 30)


## Prepare cost and prediction functions

In [29]:
def compute_hinge_loss(y, x, w):
    return np.max([0, 1 - y*(x@w)])

In [30]:
def calculate_primal_objective(y, X, w, lambda_):
    """compute the full cost (the primal objective), that is loss plus regularizer.
    X: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """
    hinge_loss = 0
    for n in range(X.shape[0]):
        hinge_loss += compute_hinge_loss(y[n], X[n, :], w)
    regularizer = lambda_*0.5*(w.T@w)
    return hinge_loss + regularizer

In [51]:
from helpers import predict_labels

def calculate_accuracy(y, X, w):
    """compute the training accuracy on the training set (can be called for test set as well).
    X: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """
    N = X.shape[0]
    wrong_predictions = 1/(2*N) * np.sum(np.abs(y - predict_labels(w, X)))  # *1/2 since abs(+1-(-1)) = 2
    accuracy = 1 - wrong_predictions
    return accuracy

## Stochastic Gradient Descent for SVM

Compute the (stochastic) subgradient for the n-th summand of the SVM optimization objective

In [33]:
def calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples):
    """compute the stochastic gradient of loss plus regularizer.
    X: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    n: the index of the (one) datapoint we have sampled
    num_examples: N
    """
    # Be careful about the constant N (size) term!
    # The complete objective for SVM is a sum, not an average as in earlier SGD examples!
    x_n, y_n = X[n], y[n]
    regularizer_gradient = lambda_*w
    z = y_n*(x_n@w)
    if z > 1:
        hinge_gradient = 0
    elif z < 1:
        hinge_gradient = num_examples*(-1)*(y_n*x_n)
    elif z == 1:
        hinge_gradient = num_examples*(-0.5)*(y_n*x_n)
    return hinge_gradient + regularizer_gradient

Implement stochastic gradient descent: Pick a data point uniformly at random and update w based on the gradient for the n-th summand of the objective

In [53]:
def sgd_for_svm_demo(y, X):
    
    max_iter = 100000
    gamma = 1
    lambda_ = 0.01
    
    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        grad = calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples)
        w -= gamma/(it+1) * grad
        
        if it % 10000 == 0:
            cost = calculate_primal_objective(y, X, w, lambda_)
            print("iteration={i}, cost={c}".format(i=it, c=cost))
    
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

sgd_for_svm_demo(y, X)

iteration=0, cost=3426254533878.605
iteration=100000, cost=52210363553.728966
iteration=200000, cost=43402829024.67824
iteration=300000, cost=38617400631.640175
iteration=400000, cost=35381021358.26699
iteration=500000, cost=32915022522.077114
iteration=600000, cost=30939932054.02576
iteration=700000, cost=29285469306.51745
iteration=800000, cost=27919428560.196938
iteration=900000, cost=26737721524.049824
training accuracy = 0.706


## Coordinate Descent (Ascent) for SVM

Compute the closed-form update for the n-th variable alpha, in the dual optimization problem, given alpha and the current corresponding w

In [101]:
def calculate_coordinate_update(y, X, lambda_, alpha, w, n):
    """compute a coordinate update (closed form) for coordinate n.
    X: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    n: the coordinate to be updated
    """
    # calculate the update of coordinate at index=n.
    x_n, y_n = X[n], y[n]
    old_alpha_n = np.copy(alpha[n])
    # Compute the update amount
    gamma_star = lambda_/(x_n.T@x_n) * (1 - y_n*w.T@x_n)
    # If alpha is one of the extremes of the interval,
    # do not update going outside the interval (it would be useless)
    if old_alpha_n == 0.0:
        gamma_star = max(gamma_star, 0)
    elif old_alpha_n == 1.0:
        gamma_star = min(gamma_star, 0)
    # Update (olny if gamma_star != 0)
    new_alpha_n = old_alpha_n + gamma_star
    # Re-project alpha inside the interval
    alpha[n] = np.clip(new_alpha_n, 0, 1)
    # Update w
    w += 1/lambda_ * (alpha[n] - old_alpha_n) * y_n * x_n
    return w, alpha

In [63]:
def calculate_dual_objective(y, X, w, alpha, lambda_):
    """calculate the objective for the dual problem."""
    return np.sum(alpha) - lambda_/2.0 * (w.T @ w)

In [102]:
def coordinate_descent_for_svm_demo(y, X):
    max_iter = 100000
    lambda_ = 0.01

    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    alpha = np.zeros(num_examples)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        w, alpha = calculate_coordinate_update(y, X, lambda_, alpha, w, n)
            
        if it % 10000 == 0:
            # primal objective
            primal_value = calculate_primal_objective(y, X, w, lambda_)
            # dual objective
            dual_value = calculate_dual_objective(y, X, w, alpha, lambda_)
            # primal dual gap
            duality_gap = primal_value - dual_value
            print('iteration=%i, primal:%.5f, dual:%.5f, gap:%.5f'%(
                    it, primal_value, dual_value, duality_gap))
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

coordinate_descent_for_svm_demo(y, X)

iteration=0, primal:6577.96823, dual:0.00000, gap:6577.96823
iteration=10000, primal:6213.82999, dual:0.00009, gap:6213.82990
iteration=20000, primal:4765.63755, dual:0.00018, gap:4765.63737
iteration=30000, primal:3243.55731, dual:0.00027, gap:3243.55703
iteration=40000, primal:6396.77154, dual:0.00036, gap:6396.77118
iteration=50000, primal:3660.78335, dual:0.00044, gap:3660.78291
iteration=60000, primal:6626.79633, dual:0.00053, gap:6626.79580
iteration=70000, primal:3392.40438, dual:0.00061, gap:3392.40377
iteration=80000, primal:5333.45896, dual:0.00070, gap:5333.45826
iteration=90000, primal:3647.97594, dual:0.00079, gap:3647.97515
training accuracy = 0.6821999999999999
